In [ ]:
!pip install praw

### Scrapping top subreddits from redditlist.com

In [ ]:
import requests
from bs4 import BeautifulSoup

npage=2
subreddits = []
for n in range(1, npage+1):
    r = requests.get(f"http://www.redditlist.com/?page={n}")
    soup = BeautifulSoup(r.text, 'lxml')
    subreddits.extend([x.text for x in soup.find(id="listing-parent").findAll(class_="span4 listing")[1].findAll("a", attrs={"class": "sfw"})])
subreddits.remove("announcements") # r/announcements is for company statements
subreddits.remove("wallstreetbets") # can't scrape r/wallstreetbets
len(subreddits)

### Getting top posts titles from subreddits

In [ ]:
import praw
import re

# pass your authentication information to Reddit instance
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    user_agent="",
    check_for_async=False,
)

rows=[]
for subreddit_name in subreddits:
    print(f'scraping {subreddit_name}')
    subreddit = reddit.subreddit(subreddit_name) 

    for post in subreddit.top(limit=None):
        title = post.title
        title = re.sub("\[.*?\]",'',title) # removing flair information from title
        title = '"'+title+'"' # wrapping inside double quotes to deal with commas in the title text
        rows.append([title, subreddit_name])

### Writing the tiles to csv file

In [ ]:
import csv

with open('posts.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["title", "label"]) # headers
    writer.writerows(rows)

print("All Done!")